# Kickstarter_survivors
### Sifan Liu

## Set up and load data

In [1]:
# Load packages and set up
import numpy as np
import pandas as pd
import json
import glob

In [2]:
## function to convert Json format when read data
def CustomParser(data):
    import json
    try:
        j1 = json.loads(data)
        return j1
    except ValueError:
        pass

In [3]:
# read data from csv 

#source:https://webrobots.io/kickstarter-datasets/
### variables in Json format
JSONconverters = {'location':CustomParser, 'category':CustomParser, 'creator':CustomParser}

## read all files in one folder
def ReadAllfiles(path):
    all_files = glob.glob(path + "/*.csv")
    df_from_each_file = (pd.read_csv(f,converters = JSONconverters) for f in all_files)
    df = pd.concat(df_from_each_file, ignore_index = True)
    return df

## loop over all folders
# paths of folders
# path2018 = r'../datasample/Kickstarter_2018-02-15T03_20_44_743Z'
path2018 = r'source/Kickstarter_2018-02-15T03_20_44_743Z'
path2017 = r'source/Kickstarter_2017-02-15T22_22_48_377Z'
path2016 = r'source/Kickstarter_2016-06-15T02_04_49_697Z'
path2015 = r'source/Kickstarter_2015-12-17T12_09_06_107Z'

paths = [path2018,path2017,path2016,path2015]

# loop over all folders to read data
files =[]
if len(paths) == 1:
    df = ReadAllfiles(paths[0]).drop_duplicates(['id']).reset_index(drop = True)
else:
    for path in paths:
        files.append(ReadAllfiles(path))
    df = pd.concat(files).drop_duplicates(['id']).reset_index(drop = True)

/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [4]:
# Check for structure
df.info()
df.state.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233923 entries, 0 to 233922
Data columns (total 37 columns):
backers_count               233923 non-null int64
blurb                       233907 non-null object
category                    233923 non-null object
converted_pledged_amount    177768 non-null float64
country                     233923 non-null object
created_at                  233923 non-null int64
creator                     233923 non-null object
currency                    233923 non-null object
currency_symbol             233923 non-null object
currency_trailing_code      233923 non-null bool
current_currency            177768 non-null object
deadline                    233923 non-null int64
disable_communication       233923 non-null bool
friends                     91 non-null object
fx_rate                     177768 non-null float64
goal                        233923 non-null float64
id                          233923 non-null int64
is_backing                  91 

successful    107609
failed        103737
canceled       14324
live            7318
suspended        935
Name: state, dtype: int64

In [5]:
# check for duplicates
df['blurb'].nunique()

231840

In [6]:
# explore why duplicated projects
df[df.duplicated(['blurb','state'], keep=False)].sort_values('blurb')

,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
150383,71,"""A pictorial calendar that will get you to ret...","{'parent_id': 18, 'position': 4, 'color': 1486...",10000.0,US,1411065455,{'avatar': {'small': 'https://ksr-ugc.imgix.ne...,USD,$,True,...,ladies-of-manure-2015-pin-up-calendar,https://www.kickstarter.com/discover/categorie...,True,True,successful,1414772402,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",10000.000000,domestic
66459,72,"""A pictorial calendar that will get you to ret...","{'slug': 'photography', 'color': 58341, 'id': ...",5135.0,US,1349976227,{'avatar': {'small': 'https://ksr-ugc.imgix.ne...,USD,$,True,...,the-ladies-of-manure-2013-calendar,https://www.kickstarter.com/discover/categorie...,True,False,successful,1355668915,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",5135.110000,domestic
114060,26,"""Book of Choices"" is a psychological drama abo...","{'parent_id': 11, 'position': 5, 'color': 1673...",5251.0,US,1405874655,{'avatar': {'small': 'https://ksr-ugc.imgix.ne...,USD,$,True,...,book-of-choices-campaign-1,https://www.kickstarter.com/discover/categorie...,True,False,successful,1409190004,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",5251.000000,domestic
116888,6,"""Book of Choices"" is a psychological drama abo...","{'parent_id': 11, 'position': 5, 'color': 1673...",2000.0,US,1413349825,{'avatar': {'small': 'https://ksr-ugc.imgix.ne...,USD,$,True,...,book-of-choices-campaign-2,https://www.kickstarter.com/discover/categorie...,True,False,successful,1416720675,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",2000.000000,domestic
48097,26,"""Central OK's comfort truck."" We will serve fr...","{'parent_id': 10, 'position': 8, 'color': 1672...",3451.0,US,1412049994,{'avatar': {'small': 'https://ksr-ugc.imgix.ne...,USD,$,True,...,from-my-tiny-kitchen-llc-food-truck,https://www.kickstarter.com/discover/categorie...,False,False,failed,1413828012,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",3451.000000,domestic
48249,20,"""Central OK's comfort truck."" We will serve fr...","{'parent_id': 10, 'position': 8, 'color': 1672...",2920.0,US,1404273501,{'avatar': {'small': 'https://ksr-ugc.imgix.ne...,USD,$,True,...,from-my-tiny-kitchen-llc,https://www.kickstarter.com/discover/categorie...,False,False,failed,1407169811,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",2920.000000,domestic
80844,1,"""Chucks Apartment"" is a High-Octane psychologi...","{'parent_id': 11, 'position': 18, 'color': 167...",1.0,US,1475803250,{'avatar': {'small': 'https://ksr-ugc.imgix.ne...,USD,$,True,...,chucks-apartment-1,https://www.kickstarter.com/discover/categorie...,False,False,failed,1480994800,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",1.000000,domestic
120313,0,"""Chucks Apartment"" is a High-Octane psychologi...","{'parent_id': 11, 'position': 9, 'color': 1673...",0.0,US,1450040722,{'avatar': {'small': 'https://ksr-ugc.imgix.ne...,USD,$,True,...,chucks-apartment-the-short-film-by-gregg-vance,https://www.kickstarter.com/discover/categorie...,False,False,failed,1454704730,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",0.000000,domestic
120299,1,"""Chucks Apartment"" is a High-Octane psychologi...","{'parent_id': 11, 'position': 9, 'color': 1673...",1.0,US,1454722209,{'avatar': {'small': 'https://ksr-ugc.imgix.ne...,USD,$,True,...,chucks-apt-the-short-film,https://www.kickstarter.com/discover/categorie...,False,False,failed,1457317633,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",1.000000,domestic
80851,1,"""Chucks Apartment"" is a High-Octane psychologi...","{'parent_id': 11, 'position': 18, 'color': 167...",1.0,US,1470603733,{'avatar': {'small': 'https://ksr-ugc.imgix.ne

In [7]:
# Parse json information to columns
def JsonConcate(dataframe, column):
    temp = dataframe[column].apply(pd.Series)
    temp = temp.rename(columns = lambda x : column + '_' + str(x))
    dataframe = pd.concat([dataframe[:],temp[:]],axis =1)
    return dataframe

# loop over columns of interests
for col in ('location','category','creator'):
    df = JsonConcate(df,col)
    print("checkpoint")

checkpoint
checkpoint
checkpoint


In [8]:
# check for data structure
df.describe()

,backers_count,converted_pledged_amount,created_at,deadline,fx_rate,goal,id,launched_at,pledged,state_changed_at,static_usd_rate,usd_pledged,location_id,category_color,category_id,category_parent_id,category_position,creator_id
count,233923.000000,1.777680e+05,2.339230e+05,2.339230e+05,177768.000000,2.339230e+05,2.339230e+05,2.339230e+05,2.339230e+05,2.339230e+05,233923.000000,2.339230e+05,2.328920e+05,2.339230e+05,233923.000000,218168.000000,233923.000000,2.339230e+05
mean,132.525771,1.065451e+04,1.408951e+09,1.415578e+09,1.017120,4.688868e+04,1.075136e+09,1.412655e+09,1.211942e+04,1.415363e+09,1.023479,1.116091e+04,3.337752e+06,1.171047e+07,142.960628,11.985704,7.382442,1.073677e+09
std,1088.863536,8.127804e+04,5.961507e+07,5.958887e+07,0.179635,1.165508e+06,6.191098e+08,5.972874e+07,1.313452e+05,5.942080e+07,0.198237,1.053898e+05,6.609745e+06,6.042961e+06,132.142615,5.168095,4.673308,6.196713e+08
min,0.000000,0.000000e+00,1.240335e+09,1.241334e+09,0.006695,1.000000e-02,1.852000e+04,1.240603e+09,0.000000e+00,1.241334e+09,0.008771,0.000000e+00,6.000000e+00,5.162700e+04,1.000000,1.000000,1.000000,3.000000e+00
25%,2.000000,6.800000e+01,1.365825e+09,1.373576e+09,1.000000,2.000000e+03,5.376027e+08,1.370863e+09,5.000000e+01,1.373565e+09,1.000000,5.000000e+01,2.357536e+06,6.526716e+06,33.000000,10.000000,4.000000,5.388774e+08
50%,18.000000,1.227000e+03,1.415857e+09,1.424023e+09,1.000000,5.000000e+03,1.078379e+09,1.421283e+09,1.016000e+03,1.423836e+09,1.000000,1.016000e+03,2.436704e+06,1.486766e+07,47.000000,12.000000,6.000000,1.073581e+09
75%,73.000000,5.545000e+03,1.453333e+09,1.459659e+09,1.000000,1.500000e+04,1.609624e+09,1.456848e+09,5.326010e+03,1.459530e+09,1.000000,5.301000e+03,2.475930e+06,1.673457e+07,285.000000,16.000000,10.000000,1.609801e+09
max,219382.000000,1.026684e+07,1.518675e+09,1.523863e+09,1.764208,1.000000e+08,2.147476e+09,1.518704e+09,2.902191e+07,1.518704e+09,1.716408,2.033899e+07,1.000000e+08,1.677606e+07,389.000000,26.000000,19.000000,2.147483e+09


## Clean Kickstarter dataset

In [9]:
# convert unix time
time_cols = ['created_at', 'deadline', 'state_changed_at','launched_at']
df[time_cols] = df[time_cols].apply(pd.to_datetime,unit='s')

In [10]:
# calculate duration
df['life'] = df['deadline'] - df['launched_at']
df['duration'] = df['state_changed_at'] - df['launched_at']
df['prep'] =  df['launched_at'] - df['created_at']

# df['state_changed_at'][0].year

In [11]:
# factorize project state
df['state'] = df['state'].astype('category')
pd.pivot_table(df, index = ['state'])

,backers_count,category_color,category_id,category_parent_id,category_position,converted_pledged_amount,creator_id,currency_trailing_code,disable_communication,fx_rate,goal,id,location_id,pledged,spotlight,static_usd_rate,usd_pledged
state,,,,,,,,,,,,,,,,,
canceled,16.730522,1.124637e+07,220.252443,11.974518,6.579657,1743.798255,1.073310e+09,0.826306,0.0,1.016327,108398.196990,1.078678e+09,3.386735e+06,2189.259002,0.0,1.021779,1871.691472
failed,15.252668,1.190604e+07,177.228703,12.477362,7.268872,1144.602144,1.074487e+09,0.852801,0.0,1.017993,73718.124368,1.076374e+09,3.392955e+06,1371.841942,0.0,1.024485,1211.500158
live,74.081853,9.305768e+06,105.424433,11.191541,6.407762,6548.369835,1.068168e+09,0.769199,0.0,0.997732,66610.450533,1.064682e+09,3.580536e+06,12016.479123,0.0,0.993897,5862.152422
successful,265.487496,1.176297e+07,101.186564,11.492964,7.676951,19341.140266,1.073367e+09,0.887398,0.0,1.017327,10381.768642,1.073785e+09,3.261350e+06,23861.596839,1.0,1.024834,22396.570157
suspended,72.659893,9.900384e+06,258.425668,12.954011,6.014973,3818.810316,1.068083e+09,0.806417,1.0,1.018514,175103.094118,1.120790e+09,3.327024e+06,6076.805604,0.0,1.013442,5704.382428


In [67]:
df['category_broad'] = df['category_slug'].str.extract(r'([^\/]+)', expand = False).astype('category')
df['category_broad'].describe()

count           321706
unique              15
top       film & video
freq             51767
Name: category_broad, dtype: object

In [13]:
# generate gender by first name
## STEP 1: strip first name
df['first_name'] = df['creator_name'].str.extract(r'([^\s]+)',expand = False).str.title()

## STEP 2: gender
#### https://pypi.org/project/gender-guesser/
import gender_guesser.detector as gender
d = gender.Detector()

## STEP 3: apply
df['gender'] = df['first_name'].apply(d.get_gender)

## Describe gender
df.gender.value_counts()

male             104812
unknown           67120
female            40373
mostly_male       10102
mostly_female      9160
andy               2356
Name: gender, dtype: int64

## Match places to MSA

In [39]:
place2msa = pd.read_csv("source/placeTOmsa.csv", encoding = 'latin-1', dtype = object)
# remove the part that's not part of the capitalized place name
# place2msa['place'] = place2msa['PLACENAME'].str.split(r'(\s)(\b[a-z]|CDP)').str[0]
place2msa['place'] = place2msa['placenm'].str.split(r'(\s)(\b[a-z]|CDP)').str[0]
place2msa.sample(15)

,state,placefp,county,stab,cntyname,placenm,pop10,afact,place
29306,48,64184,48337,TX,Montague TX,"St. Jo city, TX",1043,1,St. Jo
18667,34,99999,34003,NJ,Bergen NJ,,143244,0.041,
13003,24,99999,24023,MD,Garrett MD,,21747,0.021,
14713,27,48184,27063,MN,Jackson MN,"Okabena city, MN",188,1,Okabena
20360,36,83041,36111,NY,Ulster NY,"Woodstock CDP, NY",2088,1,Woodstock
11653,21,85008,21089,KY,Greenup KY,"Wurtland city, KY",995,1,Wurtland
9267,19,8020,19069,IA,Franklin IA,"Bradford CDP, IA",99,1,Bradford
1203,4,25650,4011,AZ,Greenlee AZ,"Franklin CDP, AZ",92,1,Franklin
28585,48,36032,48265,TX,Kerr TX,"Ingram city, TX",1804,1,Ingram
29204,48,60518,48249,TX,Jim Wells TX,"Rancho Alegre CDP, TX",1704,1,Rancho Alegre


In [40]:
df.location_state.sample(15)

93133     Stockholm
68767            FL
20986            CA
215223           ON
56982            RI
116025           NV
70152            PA
4340             NJ
139822           CA
62376            FL
147943           CA
56995       England
228073      England
26142            NY
96783            CA
Name: location_state, dtype: object

In [44]:
# match place to MSA

df = pd.merge(df, place2msa[['placefp','place','cntyname','stab','county']].drop_duplicates(), 
                 how='left', indicator=True,
                 left_on =['location_state','location_name'],
                 right_on=['stab','place'])

match_result = df['_merge'] == 'left_only'
unmatched = df[match_result]
matched = df[~match_result]


In [70]:
unmatched = pd.read_pickle("unmatched")
matched = pd.read_pickle("matched")

In [47]:
# how many places unmatched?
len(unmatched)/len(df)

0.23916868196427793

In [50]:
# creat a list with unique unmatched places
unmatched_US = unmatched.loc[(unmatched.place.isnull()) & (unmatched.location_country == 'US')]
unmatched_add = unmatched_US['location_short_name'].unique().tolist()

# how many unique unmatched places?
len(unmatched_add)

2769

In [57]:
unmatched_add

['Hollywood, CA',
 'Middletown, NJ',
 'Bloomfield, NJ',
 'Brooklyn, NY',
 'Mccloud, CA',
 'Nashville, TN',
 'Mcminnville, OR',
 'Lower East Side, NY',
 'Louisville, KY',
 'Cedar Mountain, NC',
 'Highgate Center, VT',
 'Colts Neck, NJ',
 'San Pedro, CA',
 'Ventura, CA',
 'Georgetown, MA',
 'Honolulu, HI',
 'Hamilton, NJ',
 'Bronx, NY',
 'Southington, CT',
 'Boise, ID',
 'Midlothian, VA',
 'Lexington, KY',
 'Paso Robles, CA',
 'Athens, GA',
 'Amherst, MA',
 'Bensalem, PA',
 'Long Island, NY',
 'Manhattan, NY',
 'Chelmsford, MA',
 'De Kalb, IL',
 'Trabuco, CA',
 'West Hills, CA',
 'Barrington, RI',
 'Framingham, MA',
 'North Kingstown, RI',
 'Staten Island, NY',
 'Millers Tavern, VA',
 'Sherman Oaks, CA',
 'South Florida, FL',
 'Stockbridge, MA',
 'Parsippany, NJ',
 'Denmark, ME',
 'Queens, NY',
 'De Land, FL',
 'Stafford, VA',
 'East Haddam, CT',
 'Tawas, MI',
 'Pembroke Township, IL',
 'USA, AL',
 'Hope, ME',
 'Woodbury, CT',
 'Sylmar, CA',
 'Anthony, FL',
 'South Amboy Junction, NJ',
 

### Geocoding unmatched places
Get county FIPS from FCC using lat and lng from Google Geocode API

In [51]:
import requests

def get_FCC_results(lat,lng):
    FCC_url = "https://geo.fcc.gov/api/census/block/find?latitude={}&longitude={}&format=json".format(lat,lng)
    
    results = requests.get(FCC_url)
    results = results.json()
    FIPS = results['County']['FIPS']
    
    return FIPS

In [52]:
# test
get_FCC_results("40.789142",'-73.13496099999999')

'36103'

In [53]:

def get_google_results(address, api_key = None):
    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(address)
    if api_key is not None:
        geocode_url = geocode_url + "&key={}".format(api_key)
        
    results = requests.get(geocode_url)
    results = results.json()
    if len(results['results']) == 0:
        output = {'unmatched_add':address,
                  'FIPS':None}
    else:
        answer = results['results'][0]
        lat = answer.get('geometry').get('location').get('lat')
        lng = answer.get('geometry').get('location').get('lng')
                 
        output = {'unmatched_add':address,
                  'FIPS':get_FCC_results(lat,lng)}
    
    output['status'] = results.get('status')
     
    return output

In [63]:
# test the function

get_google_results("Long Island, NY,US",API_KEY)


{'FIPS': None,
 'status': 'REQUEST_DENIED',
 'unmatched_add': 'Long Island, NY,US'}

In [11]:
# take note of error messages
import logging
import time

logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)
# create console handler
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)


In [63]:
# run api on unmatched places
results = []

for address in unmatched_add:
    geocoded = False
    while geocoded is not True:
        try:
            geocode_result = get_google_results(address, API_KEY)
        except Exception as e:
            logger.exception(e)
            logger.error("Major error with {}".format(address))
            logger.error("Skipping!")
            geocoded = True
            
        if geocode_result['status'] == 'OVER_QUERY_LIMIT':
            logger.info("Hit Query Limit at {}".format(len(results)))
            geocoded = False
        else:
            if geocode_result['status'] != 'OK':
                logger.warning("Error geocoding {}: {}".format(address, geocode_result['status']))
            results.append(geocode_result)
            geocoded = True
        
    if len(results) % 500 == 0:
        logger.info("Completed {} of {} address".format(len(results), len(unmatched_add)))


Completed 500 of 2059 address
Completed 1000 of 2059 address
Completed 1500 of 2059 address
Completed 2000 of 2059 address


In [95]:
placeMap = pd.DataFrame(results).apply(pd.Series)
placeMap.isnull().sum()

FIPS             3
status           0
unmatched_add    0
dtype: int64

In [96]:
placeMap.sample(5)

,FIPS,status,unmatched_add
1732,36079,OK,"Garrison, NY"
601,36005,OK,"South Bronx, NY"
949,28025,OK,"Cedar Bluff, MS"
545,49057,OK,"Ogden, UT"
107,09001,OK,"West Norwalk, CT"


In [116]:
county2msa = pd.read_csv("source/county2msa.csv", encoding = 'latin-1')[1:]

county2msa = pd.merge(county2msa[['county','cbsa']],
                      place2msa[['cbsa','cbsa_name','top100']].drop_duplicates(),
                      how = 'left',
                      on = 'cbsa')
county2msa.head()

,county,cbsa,cbsa_name,top100
0,01001,33860,"Montgomery, AL",0
1,01003,19300,"Daphne-Fairhope-Foley, AL",0
2,01005,21640,"Eufaula, AL-GA",0
3,01007,13820,"Birmingham-Hoover, AL",1
4,01009,13820,"Birmingham-Hoover, AL",1


In [117]:
# merge back
placeMap = pd.merge(placeMap[['unmatched_add','FIPS']], 
                    county2msa.drop_duplicates(), 
                    how = 'left', 
                    left_on=['FIPS'],
                    right_on=['county'])

placeMap.sample(5)

,unmatched_add,FIPS,county,cbsa,cbsa_name,top100
1787,"North Richland Hills, TX",48439,48439,19100,"Dallas-Fort Worth-Arlington, TX",1
1542,"West Colorado Springs, CO",08041,08041,17820,"Colorado Springs, CO",1
375,"Northridge, CA",06037,06037,31080,"Los Angeles-Long Beach-Anaheim, CA",1
1015,"Toutle, WA",53015,53015,31020,"Longview, WA",0
1534,"Biggersville, MS",28003,28003,18420,"Corinth, MS",0


In [119]:
cols_to_use = unmatched.columns.difference(placeMap.columns)

unmatched = pd.merge(placeMap,
                     unmatched[cols_to_use],
                     how = 'right',
                     left_on = ['unmatched_add'],
                     right_on = ['location_short_name'])


In [120]:
df = pd.concat([unmatched,matched])

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 321706 entries, 0 to 321705
Data columns (total 73 columns):
backers_count                321706 non-null int64
blurb                        321681 non-null object
category                     321706 non-null object
converted_pledged_amount     246176 non-null float64
country                      321706 non-null object
created_at                   321706 non-null datetime64[ns]
creator                      321706 non-null object
currency                     321706 non-null object
currency_symbol              321706 non-null object
currency_trailing_code       321706 non-null bool
current_currency             246176 non-null object
deadline                     321706 non-null datetime64[ns]
disable_communication        321706 non-null bool
friends                      106 non-null object
fx_rate                      246176 non-null float64
goal                         321706 non-null float64
id                           321706 non-null i

In [66]:
df.category_slug.sample(5)

115088                      food
297722                music/rock
193507    film & video/animation
87922                food/drinks
150740                journalism
Name: category_slug, dtype: object

## Summary: selected columns

In [3]:
df = pd.read_pickle("master")

In [68]:
# keep selected columns
col = ['location_country','location_state','location_localized_name','location_name',
       'backers_count','goal','pledged','creator_name','gender','category_broad','category_name',
       'launched_at','deadline','state_changed_at','state','life','duration','prep','slug','blurb',
      ]

df_sample = df[col]

In [123]:
# df.to_pickle("master")

In [69]:
df_sample.to_pickle("sample")
# unmatched.to_pickle("unmatched")
# matched.to_pickle("matched")